In [2]:
from keras.datasets import cifar10
import numpy as np
import cv2

(X_train_base, Y_train_base), (X_test_base, Y_test_base) = cifar10.load_data()

print("LOADED")
X_train_base = np.array([cv2.resize(img, (224, 224)) for img in X_train_base[:,:,:,:]])
print("x_train OK")
X_test_base = np.array([cv2.resize(img, (224, 224)) for img in X_test_base[:,:,:,:]])
print("x_test OK")

LOADED
x_train OK
x_test OK


In [4]:
X_train = X_train_base[:5000,:,:,:]
X_test = X_test_base[:1000,:,:,:]
Y_train = Y_train_base[:5000,:]
Y_test = Y_test_base[:1000,:]

print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

(5000, 224, 224, 3)
(5000, 1)
(1000, 224, 224, 3)
(1000, 1)


In [5]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train = np.divide(X_train, 255.0)
X_test = np.divide(X_test, 255.0)

# print(X_train[0])

In [6]:
from keras.utils import np_utils
Y_train = np_utils.to_categorical(Y_train, 10)
print(Y_train.shape)
Y_test = np_utils.to_categorical(Y_test, 10)
print(Y_test.shape)

(5000, 10)
(1000, 10)


In [7]:
from keras.layers import Conv2D, MaxPooling2D, Dense, AveragePooling2D, concatenate, Flatten, Dropout
def inception_node(data_in, filters):
    # all the conv layers' strides are (1, 1)
    
    a = Conv2D(filters, (1, 1), padding = "same", data_format = "channels_last")(data_in)

    b = Conv2D(filters, (1, 1), padding = "same", data_format = "channels_last")(data_in)
    b = Conv2D(filters, (3, 3), padding = "same", data_format = "channels_last")(b)
    
    c = Conv2D(filters, (1, 1), padding = "same", data_format = "channels_last")(data_in)
    c = Conv2D(filters, (5, 5), padding = "same", data_format = "channels_last")(c)
    
    d = MaxPooling2D((3, 3), strides = (1, 1), padding = "same")(data_in)
    d = Conv2D(filters, (1, 1), padding = "same", data_format = "channels_last")(d)
    
    merged = concatenate([a, b, c, d], axis = 3)
    
    return merged

In [8]:
from keras.activations import softmax
from keras.backend import softmax

def softmax_out(data_in):
    sf = AveragePooling2D((5, 5), strides = (3, 3), data_format = "channels_last")(data_in)
    sf = Conv2D(512, (1, 1), strides = (1, 1), padding = "same", data_format = "channels_last")(sf)
    sf = Flatten()(sf)
    sf = Dense(1024, activation = "relu")(sf)
    out = Dense(10, activation = "softmax")(sf)
    
    return out

In [ ]:
from keras.backend import int_shape
from keras.layers import Input
in_images = Input(shape = (224, 224, 3))
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization

images = Conv2D(64, (7, 7), strides = (2, 2), padding = "same", data_format = "channels_last", activation = "relu")(in_images)
print(int_shape(images))
images = MaxPooling2D((3, 3), strides = (2, 2), padding = "same")(images)
print(int_shape(images))
images = BatchNormalization()(images)
print(int_shape(images))

images = Conv2D(192, (1, 1), strides = (1, 1), padding = "same", data_format = "channels_last")(images)
print(int_shape(images))
images = Conv2D(192, (3, 3), strides = (1, 1), padding = "same", data_format = "channels_last")(images)
print(int_shape(images))
images = BatchNormalization()(images)
print(int_shape(images))

images = MaxPooling2D((3, 3), strides = (2, 2), padding = "same")(images)
print(int_shape(images))

inc_1 = inception_node(images, 256)
print(int_shape(inc_1))
inc_2 = inception_node(inc_1, 480)
print(int_shape(inc_2))
inc_2 = MaxPooling2D((3, 3), strides = (2, 2), padding = "same")(inc_2)
print(int_shape(inc_2))
inc_3 = inception_node(inc_2, 512)
print(int_shape(inc_3))
sf_out0 = softmax_out(inc_3)
print(int_shape(sf_out0))
inc_4 = inception_node(inc_3, 512)

inc_5 = inception_node(inc_4, 512)
inc_6 = inception_node(inc_5, 528)
inc_7 = inception_node(inc_6, 832)

sf_out1 = softmax_out(inc_6)

inc_8 = inception_node(inc_7, 832)
inc_9 = inception_node(inc_8, 1024)

out = AveragePooling2D((7, 7), data_format = "channels_last")(inc_9)

out = Dropout(0.4)(out)

sf_out2 = Flatten()(out)
sf_out2 = Dense(10, activation = "softmax", name = "out")(sf_out2)
print(int_shape(sf_out2))

Instructions for updating:
Colocations handled automatically by placer.


Instructions for updating:
Colocations handled automatically by placer.


(None, 112, 112, 64)
(None, 56, 56, 64)


In [70]:
from keras.models import Model
model = Model(in_images, [sf_out0, sf_out1, sf_out2])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv2d_355 (Conv2D)             (None, 112, 112, 64) 9472        input_7[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_73 (MaxPooling2D) (None, 56, 56, 64)   0           conv2d_355[0][0]                 
__________________________________________________________________________________________________
batch_normalization_13 (BatchNo (None, 56, 56, 64)   256         max_pooling2d_73[0][0]           
__________________________________________________________________________________________________
conv2d_356

In [73]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam')
model.fit(X_train, [Y_train, Y_train, Y_train], validation_data=(X_test, [Y_test, Y_test, Y_test]), epochs=50, batch_size=128)

ValueError: Input arrays should have the same number of samples as target arrays. Found 5000 input samples and 1000 target samples.